# NLP model tests
---

Here I test a few NLP models and their relevance for the project.

## Setup

### Import libraries

In [ ]:
import os
from ipywidgets import interact
import plotly.io as pio
from transformers import pipeline
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
os.chdir("..")

In [ ]:
from utils.data_utils import DATA_DIR, load_yaml_file, load_markdown_file
from utils.nlp_utils import get_sentences

### Parameters

Set the plotly style:

In [ ]:
pio.templates.default = "plotly_white"

Get the party names from the data:

In [ ]:
data_name = None
party_data = None
party_names = None
selected_party = None

In [ ]:
data_names = os.listdir(DATA_DIR)
data_names = [name for name in data_names if name != ".DS_Store"]

In [ ]:
@interact
def set_data(data=data_names):
    global data_name
    global party_data
    global party_names
    data_name = data
    party_data = load_yaml_file(os.path.join(DATA_DIR, data_name, "parties_data.yml"))
    party_names = list(party_data.keys())

Select a party:

In [ ]:
@interact
def select_party(party=party_names):
    global selected_party
    selected_party = party

### Load data

In [ ]:
program_txt = load_markdown_file(os.path.join(DATA_DIR, data_name, "programs", f"{selected_party}.md"))

In [ ]:
# sample of the text
program_txt[:100]

In [ ]:
sentences = get_sentences(program_txt)
sentences = [s.replace("*", "").replace("#", "") for s in sentences]

In [ ]:
sentences[:10]

## Test models

### Sentiment analysis

#### Load the model

In [ ]:
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=sentiment_model_path, tokenizer=sentiment_model_path)

#### Apply to all sentences

In [ ]:
sentiment_outputs = [sentiment_task(sentence) for sentence in tqdm(sentences)]

In [ ]:
sentiment_outputs[:10]

#### Get the average sentiment

In [ ]:
sentiments_dict = dict(label=[], score=[], sentence=[])
for idx, output in enumerate(sentiment_outputs):
    sentiments_dict["label"].append(output[0]["label"])
    sentiments_dict["score"].append(output[0]["score"])
    sentiments_dict["sentence"].append(sentences[idx])

In [ ]:
sentiments_df = pd.DataFrame(sentiments_dict)
sentiments_df

In [ ]:
sentiments_df[sentiments_df.label == "Negative"].sentence.sample(10).values

In [ ]:
sentiments_df[sentiments_df.label == "Positive"].sentence.sample(10).values

In [ ]:
import plotly.express as px

In [ ]:
sentiments_count = sentiments_df.label.value_counts().to_frame().reset_index()
sentiments_count.columns = ["label", "sentence_count"]
sentiments_count["percent"] = sentiments_count.sentence_count / sentiments_count.sentence_count.sum() * 100
sentiments_count["label"] = sentiments_count.label.map({"Positive": "positivo", "Negative": "negativo", "Neutral": "neutro"})
sentiments_count

In [ ]:
[float(sentiments_count[sentiments_count.label == label].percent) for label in labels_order]

In [ ]:
labels_order = ["neutro", "positivo", "negativo"]
fig = px.bar(
    x=labels_order,
    y=[float(sentiments_count[sentiments_count.label == label].percent) for label in labels_order], 
    title="Sentiment analysis",
)
fig.update_traces(
    marker_color=["gray", "green", "red"],
    hovertemplate="%{y:.1f}%<extra></extra>",
)
fig.update_layout(
    xaxis_title="Sentimento",
    yaxis_title="Percentagem de frases",
)
fig

### Hate speech

#### Load the model

In [ ]:
hate_model_path = "Hate-speech-CNERG/dehatebert-mono-portugese"
hate_task = pipeline("text-classification", model=hate_model_path, tokenizer=hate_model_path)

#### Apply to all sentences

In [ ]:
hate_outputs = [hate_task(sentence) for sentence in tqdm(sentences)]

In [ ]:
hate_outputs[:10]

#### Get the average sentiment

In [ ]:
hate_dict = dict(label=[], score=[], sentence=[])
for idx, output in enumerate(hate_outputs):
    hate_dict["label"].append(output[0]["label"])
    hate_dict["score"].append(output[0]["score"])
    hate_dict["sentence"].append(sentences[idx])

In [ ]:
hate_df = pd.DataFrame(hate_dict)
hate_df

In [ ]:
hate_df.label.value_counts()

In [ ]:
hate_df[hate_df.label == "NON_HATE"].sentence.sample(10).values

In [ ]:
hate_df[hate_df.label == "HATE"].sentence.sample(10).values

In [ ]:
hate_df[hate_df.label == "HATE"].score.describe()

In [ ]:
hate_df[(hate_df.label == "HATE") & (hate_df.score > 0.7)].sentence.values

In [ ]:
hate_df[(hate_df.label == "HATE") & (hate_df.score > 0.7)]

In [ ]:
hate_df["label"] = hate_df.apply(lambda row: "HATE" if ((row.label == "HATE") & (row.score > 0.8)) else "NON_HATE", axis=1)

In [ ]:
hate_count = hate_df.label.value_counts().to_frame().reset_index()
hate_count.columns = ["label", "sentence_count"]
hate_count["percent"] = hate_count.sentence_count / hate_count.sentence_count.sum() * 100
hate_count["label"] = hate_count.label.map({"HATE": "ódio", "NON_HATE": "neutro"})
hate_count

In [ ]:
# TODO Plot

### Summarization

#### Load the model

In [ ]:
sum_model_path = "csebuetnlp/mT5_multilingual_XLSum"
sum_task = pipeline("text2text-generation", model=sum_model_path, tokenizer=sum_model_path)

#### Apply to the whole program

In [ ]:
program_summary = sum_task(program_txt.replace("*", "").replace("#", ""))
program_summary

#### Apply to chapters